Cab Trips Stat

Initial setup

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import json
import datetime
import plotly
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

fig = make_subplots(rows=8, cols=1, subplot_titles=("Number of Trips","Cost Per Km", "Cost Per KM (Avg)", "Money Spent", "Distance Travelled"))

fig2 = make_subplots(rows=8, cols=1, subplot_titles=("Stats"))

data_base_dir = "<Directory of the file>"

First I want to read the rider data into a datafarame

In [ ]:
fileDir = data_base_dir + "/Rider/trips_data.csv"
ride_data = pd.read_csv(fileDir)

Top level Stats
Here I want to work with only the completed trips to find top level stats such as:

1. Total Amount
2. Average Amount Per Trip
3. Total Distance Travelled
4. Avergage distance tarvelled per ride.
5. Highest Amount Paid
6. Longest Ride (Distance)
7. Longest Ride (Duration)
8. Avg cost per km
9. Percentage of trips cancelled


First we will filter the data by completed trips



In [ ]:
is_paid = ride_data['Fare Amount'] > 0
paid_trips = ride_data[is_paid]
is_completed = paid_trips['Trip or Order Status'] == "COMPLETED"
trips = paid_trips[is_completed]
filter_service = trips['Product Type'] != "UberEATS Marketplace"

completed_trips = trips[filter_service]
completed_trips = completed_trips.reset_index(drop=True)

completed_trips['Product Type'] = completed_trips['Product Type'].str.upper()

number_of_completed_trips = len(completed_trips)

request_date = []
trip_start = []
trip_end= []
duration = []
month_s=[]
year_s=[]
month_year_s = []
cost_per_km_s = []

size = len(completed_trips)
for i in range(size):
    
    date = datetime.datetime.strptime(completed_trips['Request Time'][i][:-19],'%Y-%m-%d')
    start = datetime.datetime.strptime(completed_trips['Begin Trip Time'][i][:-10],'%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(completed_trips['Dropoff Time'][i][:-10],'%Y-%m-%d %H:%M:%S')
    month = date.month
    year = date.year
    month_year = datetime.datetime.strptime(completed_trips['Request Time'][i][:-22],'%Y-%m')
    cost_per_km = completed_trips['Fare Amount'][i] / (completed_trips['Distance (miles)'][i] * 1.6)
    
    request_date.append(date)
    trip_start.append(start)
    trip_end.append(end)
    d = end-start
    duration.append(d)
    month_s.append(month)
    year_s.append(year)
    month_year_s.append(month_year)
    cost_per_km_s.append(cost_per_km)

    
    #print("{0}, {1}, {2}".format(date, start, d))

completed_trips['trip_start'] = trip_start
completed_trips['request_date'] = request_date
completed_trips['trip_end'] = trip_end
completed_trips['duration']  = duration
completed_trips['month']  = month_s
completed_trips['year']  = year_s
completed_trips['month_year']  = month_year_s
completed_trips['cost_per_km']  = cost_per_km_s


trips_by_date = completed_trips.groupby(['request_date']).size().reset_index(name='count')
trips_by_month = completed_trips.groupby(['month']).size().reset_index(name='count')
trips_by_year = completed_trips.groupby(['year']).size().reset_index(name='count')
trips_by_month_year = completed_trips.groupby(['month_year']).size().reset_index(name='count')
trips_by_product_type = completed_trips.groupby(['Product Type']).size().reset_index(name='count') 

In [ ]:
total_amount = completed_trips['Fare Amount'].sum()
average_amount = format((total_amount / number_of_completed_trips),'.2f')
total_distance_km = completed_trips['Distance (miles)'].sum() * 1.6
average_distance_per_trip = total_distance_km / number_of_completed_trips
total_distance_km_str = format(total_distance_km,'.2f')
average_distance_per_trip_str = format(average_distance_per_trip,'.2f')
most_expensive_ride = completed_trips['Fare Amount'].max(skipna = True)
cheapest_ride = completed_trips['Fare Amount'].min(skipna = True)
longest_ride_distance= completed_trips['Distance (miles)'].max(skipna = True)*1.6
shortest_ride_distance= completed_trips['Distance (miles)'].min(skipna = True)*1.6
longest_ride_distance_str = format(longest_ride_distance,'0.2f')
shortest_ride_distance_str = format(shortest_ride_distance,'0.2f')
average_cost_per_km =  total_amount / total_distance_km
average_cost_per_km_str = format(average_cost_per_km,'.2f')
first_cab = completed_trips['trip_start'].min()
last_cab = completed_trips['trip_start'].max()
longest_ride_time = completed_trips['duration'].max(skipna = True)
shortest_ride_time = completed_trips['duration'].min(skipna = True)
total_duration = completed_trips['duration'].sum()
average_duration = total_duration / number_of_completed_trips
maximum_trips_in_a_day= trips_by_date['count'].max(skipna = True)


print("Total trips: {0} ".format(number_of_completed_trips))
print("First Cab Taken On: {0}".format(first_cab))
print("Last Cab Taken On: {0}".format(last_cab))
print("Max bookings in a day : {0}".format(maximum_trips_in_a_day))

print("\nMoney:")
print("Total Amount : Rs.{0}/-".format(total_amount))
print("Average Amount Per Trip : Rs.{0}/-".format(average_amount))
print("Most Expensive Ride : Rs.{0}/-".format(most_expensive_ride))
print("Cheapest Ride : Rs.{0}/-".format(cheapest_ride))
print("Avrage Cost Per Km : Rs.{0}/-".format(average_cost_per_km_str))

print("\nDistance & Time:")
print("Total Distance Travelled in cab: {0} Kms".format(total_distance_km_str))
print("Average Distance Travelled Per Trip : {0} Kms".format(average_distance_per_trip_str))
print("Longest (Distance) Ride : {0} Kms hrs".format(longest_ride_distance_str))
print("Shortest (Distance) Ride : {0} Kms".format(shortest_ride_distance_str))
print("Total Time Spent in cab : {0} hrs".format(total_duration))
print("Longest (Time) Ride : {0}".format(longest_ride_time))
print("Average (Time) Per Ride : {0} hrs".format(average_duration))


And I want to create timeseries data to view time based trends such as :

1. Number of rides per month
2. Average money spent per month
3. Average distance per month

In [ ]:
avg_cost_per_km_year = completed_trips.groupby('year')['cost_per_km'].mean().reset_index(name='Avg_Cost_Per_Km')
avg_cost_per_km_month_year = completed_trips.groupby('month_year')['cost_per_km'].mean().reset_index(name='Avg_Cost_Per_Km')
money_spent_per_year= completed_trips.groupby('year')['Fare Amount'].sum().reset_index(name='Money_Spent')
distance_per_year = completed_trips.groupby('year')['Distance (miles)'].sum().reset_index(name='total_distance')
distance_per_year['total_distance'] = distance_per_year['total_distance'] * 1.6

In [ ]:
import plotly.graph_objects as go

values = [['Total trips', 
           'First Cab Taken On', 
           'Last Cab Taken On', 
           'Max bookings in a day',
           '<b>Total Amount</b>', 
            'Average Amount Per Trip',
           'Most Expensive Ride', 
            'Cheapest Ride',
           'Avrage Cost Per Km', 
           '<b> Total Distance Travelled in Cab </b>', 
           'Average Distance Travelled Per Trip ', 
           'Longest (Distance) Ride',
           'Shortest (Distance) Ride', 
           '<b>Total Time Spent in Cab</b>', 
           'Longest (Time) Ride', 
            'Average (Time) Per Ride',
          ], #1st col
  [number_of_completed_trips, 
    first_cab, 
    last_cab, 
    maximum_trips_in_a_day,
    total_amount,
    average_amount,
    most_expensive_ride,
    cheapest_ride,
    average_cost_per_km_str,
    total_distance_km_str,
    average_distance_per_trip_str,
    longest_ride_distance_str,
    shortest_ride_distance_str,
    str(total_duration),
    str(longest_ride_time),
    str(average_duration)]]


fig2 = go.Figure(data=[go.Table(
  columnorder = [1,2],
  columnwidth = [60,50],
  header = dict(
    values = [['<b>DATA STATS</b>'],
                  ['<b>VALUE</b>']],
    line_color='darkslategray',
    fill_color='royalblue',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['paleturquoise', 'white']),
    align=['left', 'right'],
    font_size=12,
    height=30)
    )
])

fig2.show()

In [ ]:
#print(completed_trips[completed_trips['month_year'] == "2017-08-01"][['month_year', 'cost_per_km']])
#print(trips_by_month)
#print(trips_by_year)
#  - plotly.graph_objs.scatter.Line
#  - plotly.graph_objs.layout.shape.Line


fig.add_trace(
   go.Bar(x=trips_by_month_year['month_year'], y=trips_by_month_year['count'],name="Trips per month"),
    row=1, col=1,
)
fig.add_trace(
   go.Line(x=trips_by_year['year'], y=trips_by_year['count'],name="Trips per year"),
    row=1, col=1,
)
fig.add_trace(
   go.Line(x=completed_trips['trip_start'], y=completed_trips['cost_per_km'],name="Cost Per Km"),
    row=2, col=1,
)
fig.add_trace(
   go.Line(x=avg_cost_per_km_month_year['month_year'], y=avg_cost_per_km_month_year['Avg_Cost_Per_Km'],name="Avg_Cost_Per_Km"),
    row=3, col=1,
)
fig.add_trace(
   go.Line(x=avg_cost_per_km_year['year'], y=avg_cost_per_km_year['Avg_Cost_Per_Km'],name="Avg_Cost_Per_Km"),
    row=3, col=1,
)

fig.add_trace(
   go.Line(x=money_spent_per_year['year'], y=money_spent_per_year['Money_Spent'],name="Money_Spent"),
    row=4, col=1,
)

fig.add_trace(
   go.Line(x=distance_per_year['year'], y=distance_per_year['total_distance'],name="Distance Travelle"),
    row=5, col=1,
)

fig.add_trace(
   go.Line(x=distance_per_year['year'], y=distance_per_year['total_distance'],name="Distance Travelle"),
    row=5, col=1,
)
fig.add_trace(
   go.Bar(x=trips_by_product_type['Product Type'], y=trips_by_product_type['count'],name="Product Type"),
    row=6, col=1,
)


#fig.add_trace(
 #   go.Pie(values=trips_by_product_type['Product Type']),
  #            row=8, col=2,
#)



In [ ]:
fig2.update_layout(height=800, width=1200, title_text="Stats: " +data_base_dir, legend_orientation="h")
fig2.show()
fig.update_layout(height=2500, width=1200, title_text="Stats: " +data_base_dir, legend_orientation="h")
fig.show()
